In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
%matplotlib inline
sns.set_style('darkgrid')

In [ ]:
#%% [code]
###COVID DATA
## Retreving Data
# Assigning Github URLs
cases_url="https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
deaths_url="https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
# recovered_url="https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"

# Reading URLs through Pandas liberary
total_cases = pd.read_csv(cases_url)
total_deaths = pd.read_csv(deaths_url)
# total_recovered = pd.read_csv(recovered_url)

## Cleaning Data
# Extreacting and storing column dates in dates
dates = total_cases.columns[4:]

# Reshaping total cases column from wide shape to df shape
total_cases_df = total_cases.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Confirmed'
    )

# Reshaping total deaths column from wide shape to df shape
total_deaths_df = total_deaths.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Deaths'
    )

# # Reshaping total recovered column from wide shape to df shape
# total_recovered_df = total_recovered.melt(
#     id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
#     value_vars=dates, 
#     var_name='Date', 
#     value_name='Recovered'
# )

# Merging total_cases_df and total_deaths_df
covid_df = total_cases_df.merge(
  right=total_deaths_df, 
  how='left',
  on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long']
)

# # Merging covid_df and total_recovered_df
# covid_df = covid_df.merge(
#   right=total_recovered_df, 
#   how='left',
#   on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long']
# )

# Converting Date to proper datetime format
covid_df["Date"] = pd.to_datetime(covid_df["Date"])

# Inserting converted date to full table
covid_df = covid_df.sort_values(['Country/Region', 'Date'])
#covid_df = covid_df.sort_values(['Country/Region', 'Date'], ascending=[True, True])
# Reseting Index
# covid_df = covid_df.reset_index()

# Copying covid_df to covid_df
#covid_df = covid_df.copy()

# Renaming column names
covid_df = covid_df.rename(columns={'Country/Region': 'country',
                                    'Province/State':'province',
                                    'Confirmed': 'total_cases',
                                    'Deaths': 'total_deaths',
                                    'Date':'date'})

# Filling NaN value of provinve with country name and storing it in coiuntry column
covid_df["province"] = covid_df["province"].fillna(covid_df["country"])
covid_df["country"] = covid_df["province"]

# Adding New cases, New deaths, New recovered to covid_df
covid_df["daily_cases"] = covid_df.groupby('country')['total_cases'].diff().fillna(0)
covid_df["daily_deaths"] = covid_df.groupby('country')['total_deaths'].diff().fillna(0)
# covid_df["New recovered"] = covid_df.groupby('country')['Total recovered'].diff().fillna(0)

# Converting float value to int in covid_df columns
# covid_df["Total recovered"] = covid_df["Total recovered"].fillna(0)
# covid_df["Total recovered"] = covid_df["Total recovered"].astype(int).fillna(0)
covid_df["daily_cases"] = covid_df["daily_cases"].astype(int)
covid_df["daily_deaths"] = covid_df["daily_deaths"].astype(int)
# covid_df["New recovered"] = covid_df["New recovered"].astype(int)

# Arranging Columns
covid_df = covid_df[['date', 'province', 'country','Lat','Long',
 'total_cases', 'total_deaths', 'daily_cases','daily_deaths']]

# Removind unnecessary columns
covid_df = covid_df.drop(['province', 'Lat', 'Long'],axis=1)

# Removing unnecessary spaces and making all columns in lowercase
covid_df.country = covid_df.country.str.lower().str.strip()

# Reseting jumbled index in order
covid_df = covid_df.reset_index(drop=True)


In [ ]:
###VACCINE DATA
## Atribuindo URLs do GitHub
vaccine_data = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations.csv')
vaccine_data['date']=pd.to_datetime(vaccine_data.date)

#Mudança no formato de dados
vaccine_data['date'] = pd.to_datetime(vaccine_data['date'],format='%y-%m-%d').dt.date

#Descartando os dados desnecessários para a analise
vaccine_data.drop(['total_vaccinations_per_hundred',
                   'iso_code',
                   'people_vaccinated_per_hundred',
                   'people_fully_vaccinated_per_hundred',
                   'daily_vaccinations_per_million',
                   'daily_vaccinations_raw',
                   'total_vaccinations',
                   'people_vaccinated',
                   'daily_people_vaccinated_per_hundred',
                   'total_boosters_per_hundred'], inplace=True, axis=1)


#Adicionando a coluna de vacinação total 
Total_vaccinations = vaccine_data.groupby('location')['daily_vaccinations'].cumsum()
vaccine_data.insert(5,'Total_vaccinations',Total_vaccinations)

#Renomeando as colunas
vaccine_df = vaccine_data.rename(columns={'location': 'country',
                                    # 'date':'Date',
                                    'people_fully_vaccinated': 'fully_vaccinated',
                                    # 'total_boosters': 'Total boosters',
                                    'Total_vaccinations':'total_vaccinations',
                                    # 'daily_vaccinations': 'Daily vaccinations',
                                    # 'daily_people_vaccinated': 'Daily people vaccinated'})
                                    })
#Organizando as colunas
vaccine_df = vaccine_df[['date',
                         'country',
                         "fully_vaccinated",
                         'total_vaccinations',
                         'total_boosters',
                         'daily_vaccinations',
                         'daily_people_vaccinated']]

# Convertendo a data para o formato de data e hora convencionais
vaccine_df["date"] = pd.to_datetime(vaccine_df["date"])
vaccine_df = vaccine_df.fillna(0)
vaccine_df = vaccine_df.astype({"fully_vaccinated":'int',
                                'total_vaccinations':'int',
                                'total_boosters':'int',
                                'daily_vaccinations':'int',
                                'daily_people_vaccinated':'int'})

# Removendo espaços desnecessários  
vaccine_df.country = vaccine_df.country.str.lower().str.strip()

In [ ]:
df = pd.merge(covid_df, vaccine_df, how="outer", on=["date", "country"])
df.fillna(0,inplace=True)

In [ ]:
def plot_treemap(col):
    fig = px.treemap(df, path=["country"], values=col, height=700,
                 title=col, color_discrete_sequence = px.colors.qualitative.Dark2)
    fig.data[0].textinfo = 'label+text+value'
    fig.show()

# Total de casos de covid em cada país do mundo

In [ ]:
plot_treemap('total_cases')

# Total de vacinações por país 


In [ ]:
plot_treemap('total_vaccinations')

# Total de mortes por covid em cada país 

In [ ]:
plot_treemap('total_deaths')

In [ ]:
world = df[df.country==''].tail(1)
con = list(df.country.unique())
for x in con:
  world = world.append(df[df.country==x].tail(1), ignore_index=True)
world

In [ ]:
def plot_hbar(df, col, n, hover_data=[]):
  
    fig = px.bar(df.sort_values(col).tail(n), 
                 x=col, y="country", color='country',  
                 text=col, orientation='h', width=700, hover_data=hover_data,
                 color_discrete_sequence = px.colors.qualitative.Dark2)
    
    fig.update_layout(title=col, xaxis_title="", yaxis_title="", 
                      yaxis_categoryorder = 'total ascending',
                      uniformtext_minsize=8, uniformtext_mode='hide')
    
    fig.show()

# Os 15 países com mais casos de covid no mundo

In [ ]:
plot_hbar(world, 'total_cases', 15)

# Usando o Brasil como exemplo de analise

In [ ]:
fig = px.line(df[df.country=='brazil'],x='date', y=df[df.country=='brazil'].columns[2], title="Casos de covid no Brasil")
fig.show()

In [ ]:
fig = px.line(df[df.country=='brazil'],x='date', y=df[df.country=='brazil'].columns[9:11], title="Vacinação na Brasil")
fig.show()


In [ ]:
fig = px.line(df[df.country=='brazil'],x='date', y=df[df.country=='brazil'].columns[2:8],title="Dados do Brasil")
fig.show()